In [18]:
import numpy as np 
import pandas as pd
from pathlib import Path
import glob

In [19]:
# Folder that contains all the Streaming_History_Audio_*.json files
FOLDER = r"C:\Users\josep\Downloads\Winter 2026\jukejam\data\raw\Personal Data\Joseph\Spotify Extended Streaming History"

# Find all audio history json files (adjust pattern if needed)
pattern = str(Path(FOLDER) / "Streaming_History_Audio_*.json")
files = sorted(glob.glob(pattern))

# Load + combine
dfs = [pd.read_json(f) for f in files]
spotify_events = pd.concat(dfs, ignore_index=True)

print("Combined shape:", spotify_events.shape)
spotify_events.head(5)


Combined shape: (189270, 23)


,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2021-07-11T16:20:48Z,"iOS 14.6 (iPhone9,3)",3221,US,76.90.107.12,Slumber Party (feat. Princess Nokia),Ashnikko,DEMIDEVIL,spotify:track:11ZulcYY4lowvcQm4oe3VJ,None,...,NaN,NaN,NaN,clickrow,fwdbtn,False,False,False,NaN,False
1,2021-07-11T16:20:49Z,"iOS 14.6 (iPhone9,3)",1045,US,76.90.107.12,I Like Him,Princess Nokia,Everything Sucks,spotify:track:68UU9oaQtMDnh6PRlW842H,None,...,NaN,NaN,NaN,fwdbtn,fwdbtn,False,False,False,NaN,False
2,2021-07-11T16:21:23Z,"iOS 14.6 (iPhone9,3)",19050,US,76.90.107.12,It's Not My Fault,Princess Nokia,It's Not My Fault,spotify:track:3P27Dj2AqFIyy6ExlstOVi,None,...,NaN,NaN,NaN,fwdbtn,endplay,False,False,False,NaN,False
3,2021-07-11T16:21:25Z,"iOS 14.6 (iPhone9,3)",1578,US,76.90.107.12,S.I.M.P,Kylesimps,S.I.M.P,spotify:track:1FqLXcLdKdXBGh9mY9wLu4,None,...,NaN,NaN,NaN,playbtn,endplay,False,False,False,NaN,False
4,2021-07-11T16:21:26Z,"iOS 14.6 (iPhone9,3)",768,US,76.90.107.12,S.I.M.P. (Squirrels in My Pants),2 Guys n the Parque,Phineas and Ferb,spotify:track:76Wrq73y3Ef7VNM4CGSZ0l,None,...,NaN,NaN,NaN,playbtn,endplay,False,False,False,NaN,False


In [20]:
spotify_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189270 entries, 0 to 189269
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ts                                 189270 non-null  object 
 1   platform                           189270 non-null  object 
 2   ms_played                          189270 non-null  int64  
 3   conn_country                       189270 non-null  object 
 4   ip_addr                            189270 non-null  object 
 5   master_metadata_track_name         189187 non-null  object 
 6   master_metadata_album_artist_name  189187 non-null  object 
 7   master_metadata_album_album_name   189187 non-null  object 
 8   spotify_track_uri                  189187 non-null  object 
 9   episode_name                       79 non-null      object 
 10  episode_show_name                  79 non-null      object 
 11  spotify_episode_uri                79 n

In [21]:
KEEP_COLS = [
    "ts", "ms_played", "spotify_track_uri", "skipped",
    "shuffle", "platform", "reason_end",
    "conn_country"   
]

events = spotify_events[spotify_events["spotify_track_uri"].notna()].copy()
KEEP_COLS = [c for c in KEEP_COLS if c in events.columns]
events = events[KEEP_COLS].copy()

events.head(3)

,ts,ms_played,spotify_track_uri,skipped,shuffle,platform,reason_end,conn_country
0,2021-07-11T16:20:48Z,3221,spotify:track:11ZulcYY4lowvcQm4oe3VJ,False,False,"iOS 14.6 (iPhone9,3)",fwdbtn,US
1,2021-07-11T16:20:49Z,1045,spotify:track:68UU9oaQtMDnh6PRlW842H,False,False,"iOS 14.6 (iPhone9,3)",fwdbtn,US
2,2021-07-11T16:21:23Z,19050,spotify:track:3P27Dj2AqFIyy6ExlstOVi,False,False,"iOS 14.6 (iPhone9,3)",endplay,US


In [22]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Index: 189187 entries, 0 to 189269
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ts                 189187 non-null  object
 1   ms_played          189187 non-null  int64 
 2   spotify_track_uri  189187 non-null  object
 3   skipped            189187 non-null  bool  
 4   shuffle            189187 non-null  bool  
 5   platform           189187 non-null  object
 6   reason_end         189187 non-null  object
 7   conn_country       189187 non-null  object
dtypes: bool(2), int64(1), object(5)
memory usage: 10.5+ MB


In [23]:
events["timestamp"] = pd.to_datetime(events["ts"], utc=True, errors="coerce")
events = events.dropna(subset=["timestamp"])


In [24]:
events["timestamp"]

0        2021-07-11 16:20:48+00:00
1        2021-07-11 16:20:49+00:00
2        2021-07-11 16:21:23+00:00
3        2021-07-11 16:21:25+00:00
4        2021-07-11 16:21:26+00:00
                    ...           
189265   2025-07-10 08:47:37+00:00
189266   2025-07-10 08:50:43+00:00
189267   2025-07-10 08:54:05+00:00
189268   2025-07-10 08:57:09+00:00
189269   2025-07-10 09:02:01+00:00
Name: timestamp, Length: 189187, dtype: datetime64[ns, UTC]

In [25]:
events["spotify_id"] = events["spotify_track_uri"].str.replace("spotify:track:", "", regex=False)


In [26]:
events["spotify_id"]

0         11ZulcYY4lowvcQm4oe3VJ
1         68UU9oaQtMDnh6PRlW842H
2         3P27Dj2AqFIyy6ExlstOVi
3         1FqLXcLdKdXBGh9mY9wLu4
4         76Wrq73y3Ef7VNM4CGSZ0l
                   ...          
189265    2nTbWfwXlp9gK5u205Pcm1
189266    1mHD6G4fg3Xv5GHPbp1YPx
189267    2MfEPa673rIPzBUdwfqd0P
189268    3lIgiul24q1uwE8l928s0w
189269    5gf0PsMTswpUXqtRaM96aF
Name: spotify_id, Length: 189187, dtype: object

In [27]:
def time_bucket(dt):
    h = dt.hour
    if 5 <= h < 12: return "morning"
    if 12 <= h < 18: return "afternoon"
    return "night"

# Use local-time-like buckets (remove timezone)
events["time_of_day"] = events["timestamp"].dt.tz_convert(None).apply(time_bucket)


In [28]:
events["time_of_day"]

0         afternoon
1         afternoon
2         afternoon
3         afternoon
4         afternoon
            ...    
189265      morning
189266      morning
189267      morning
189268      morning
189269      morning
Name: time_of_day, Length: 189187, dtype: object

In [29]:
# Step: Create event_type column
events["event_type"] = np.where(events["skipped"], "SKIP", "PLAY")
events.loc[events["reason_end"] == "endplay", "event_type"] = "COMPLETE"

events["event_type"].value_counts()

event_type
PLAY        95056
SKIP        77430
COMPLETE    16701
Name: count, dtype: int64

In [30]:
# Create user_events dataset (without song features)
user_events_cols = [
    "timestamp", "spotify_id",
    "ms_played", "skipped", "shuffle",
    "platform", "reason_end",
    "time_of_day", "event_type"
]
if "conn_country" in events.columns:
    user_events_cols.append("conn_country")

user_events = events[user_events_cols].copy()
print("user_events shape:", user_events.shape)
user_events.head(3)

user_events shape: (189187, 10)


,timestamp,spotify_id,ms_played,skipped,shuffle,platform,reason_end,time_of_day,event_type,conn_country
0,2021-07-11 16:20:48+00:00,11ZulcYY4lowvcQm4oe3VJ,3221,False,False,"iOS 14.6 (iPhone9,3)",fwdbtn,afternoon,PLAY,US
1,2021-07-11 16:20:49+00:00,68UU9oaQtMDnh6PRlW842H,1045,False,False,"iOS 14.6 (iPhone9,3)",fwdbtn,afternoon,PLAY,US
2,2021-07-11 16:21:23+00:00,3P27Dj2AqFIyy6ExlstOVi,19050,False,False,"iOS 14.6 (iPhone9,3)",endplay,afternoon,COMPLETE,US


In [31]:
# Load songs_min catalog (created in cleanData.ipynb)
songs_min_path = Path("../data/processed/songs_min.csv")
songs_min = pd.read_csv(songs_min_path)
print("songs_min shape:", songs_min.shape)
songs_min.head(2)

songs_min shape: (50674, 11)


,spotify_id,title,artist_name,genres,duration_ms,release_year,tempo_bpm,energy,danceability,acousticness,valence
0,09ZQ5TmUG8TSL56n0knqrj,Mr. Brightside,The Killers,"rock, alternative, indie, alternative_rock, in...",222200,2004,148.114,0.918,0.355,0.001190,0.240
1,06UfBBDISthj1ZJAtX4xjj,Wonderwall,Oasis,"rock, alternative, indie, pop, alternative_roc...",258613,2006,174.426,0.892,0.409,0.000807,0.651


In [32]:
# Create enriched dataset by merging user_events with song features
enriched = user_events.merge(songs_min, on="spotify_id", how="left")

match_rate = enriched["title"].notna().mean() * 100
print(f"Match rate with songs_min: {match_rate:.2f}%")
print(f"Enriched shape: {enriched.shape}")
enriched.head(3)

Match rate with songs_min: 0.19%
Enriched shape: (189187, 20)


,timestamp,spotify_id,ms_played,skipped,shuffle,platform,reason_end,time_of_day,event_type,conn_country,title,artist_name,genres,duration_ms,release_year,tempo_bpm,energy,danceability,acousticness,valence
0,2021-07-11 16:20:48+00:00,11ZulcYY4lowvcQm4oe3VJ,3221,False,False,"iOS 14.6 (iPhone9,3)",fwdbtn,afternoon,PLAY,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-07-11 16:20:49+00:00,68UU9oaQtMDnh6PRlW842H,1045,False,False,"iOS 14.6 (iPhone9,3)",fwdbtn,afternoon,PLAY,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-07-11 16:21:23+00:00,3P27Dj2AqFIyy6ExlstOVi,19050,False,False,"iOS 14.6 (iPhone9,3)",endplay,afternoon,COMPLETE,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Select final columns for user_events_enriched
enriched_cols = [
    "timestamp", "spotify_id",
    "event_type", "ms_played", "skipped",
    "shuffle", "platform", "reason_end", "time_of_day",
    "title", "artist_name", "genres",
    "duration_ms", "release_year",
    "tempo_bpm", "energy", "danceability", "acousticness", "valence"
]
if "conn_country" in enriched.columns:
    enriched_cols.insert(9, "conn_country")  # insert after time_of_day

enriched_cols = [c for c in enriched_cols if c in enriched.columns]
user_events_enriched = enriched[enriched_cols].copy()

print("user_events_enriched shape:", user_events_enriched.shape)
user_events_enriched.head(3)

user_events_enriched shape: (189187, 20)


,timestamp,spotify_id,event_type,ms_played,skipped,shuffle,platform,reason_end,time_of_day,conn_country,title,artist_name,genres,duration_ms,release_year,tempo_bpm,energy,danceability,acousticness,valence
0,2021-07-11 16:20:48+00:00,11ZulcYY4lowvcQm4oe3VJ,PLAY,3221,False,False,"iOS 14.6 (iPhone9,3)",fwdbtn,afternoon,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-07-11 16:20:49+00:00,68UU9oaQtMDnh6PRlW842H,PLAY,1045,False,False,"iOS 14.6 (iPhone9,3)",fwdbtn,afternoon,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-07-11 16:21:23+00:00,3P27Dj2AqFIyy6ExlstOVi,COMPLETE,19050,False,False,"iOS 14.6 (iPhone9,3)",endplay,afternoon,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# Save both datasets
out_dir = Path("../data/processed")
out_dir.mkdir(parents=True, exist_ok=True)

# Save user_events (no song features)
user_events.to_csv(out_dir / "user_events.csv", index=False)
print(f"Saved: {out_dir / 'user_events.csv'} | rows: {len(user_events)}")

# Save user_events_enriched (with song features)
user_events_enriched.to_csv(out_dir / "user_events_enriched.csv", index=False)
print(f"Saved: {out_dir / 'user_events_enriched.csv'} | rows: {len(user_events_enriched)}")

Saved: ..\data\processed\user_events.csv | rows: 189187
Saved: ..\data\processed\user_events_enriched.csv | rows: 189187


In [35]:
# Sanity checks
print("=== user_events ===")
print("Unique tracks:", user_events["spotify_id"].nunique())
print("Skip rate:", round(user_events["skipped"].mean() * 100, 2), "%")
print("Platforms:", user_events["platform"].value_counts().head(5).to_dict())
print()
print("=== user_events_enriched ===")
print("Matched tracks (has title):", user_events_enriched["title"].notna().sum())
print("Unmatched tracks:", user_events_enriched["title"].isna().sum())
print("Event types:", user_events_enriched["event_type"].value_counts().to_dict())

=== user_events ===
Unique tracks: 22792
Skip rate: 47.97 %
Platforms: {'ios': 144862, 'iOS 14.8 (iPhone9,4)': 29182, 'iOS 14.7.1 (iPhone9,4)': 6239, 'windows': 2665, 'iOS 14.6 (iPhone9,3)': 1822}

=== user_events_enriched ===
Matched tracks (has title): 351
Unmatched tracks: 188836
Event types: {'PLAY': 95056, 'SKIP': 77430, 'COMPLETE': 16701}
